例37是调用的百度AI通用文字识别来识别图片上的文字。但工作中有很多表格文档的扫描件，有没有能直接识别表格的模块，识别完成后直接存为Excel表格呢？答案是：有的，老板！那就是百度人工智能的`tableRecognitionAsync`接口。接口说明网址：https://cloud.baidu.com/doc/OCR/s/3k3h7yeqa 。今天我们就来试试用Python调用表格识别模块识别表格文档的扫描件吧。在开始前，需要检查图片的大小，必须在1K~4M之间，不然会报错“'error_msg': 'image size error, image is too big or too small, upper limit 4M, lower limit 1k, please check your param'”。

In [5]:
from aip import AipOcr #导入AipOcr模块，用于做文字识别

APP_ID = '你申请的'
API_KEY = '你申请的'
SECRET_KEY = '你申请的'
client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

file = "图片\\3.png"
pic = open(file,'rb') #以二进制方式打开图片
img = pic.read() #读取
table = client.tableRecognitionAsync(img)    #调用表格识别模块
print(table)

{'result': [{'request_id': '20109216_1823419'}], 'log_id': 1590807976477153}


我们先随便取一张图片来试运行一下。最开始，还是老规矩，导入AipOcr模块，填好在百度AI平台申请的ID和KEY。然后以二进制方式打开图片，读取，然后调用表格识别模块`tableRecognitionAsync()`，并将返回值存入变量`table`中。打印`table`，看它长啥样儿。我们看到`table`是一个字典，其中有2个键，一个叫`result`，一个叫`log_id`。我们需要的是`result`中的`request_id`，通过这个ID，可以获取识别完成后所保存的Excel表格的下载地址。提取`request_id`的方法如下。因为`result`键对应的值是一个列表，列表里面只有1个元素，所以需要加上`[0]`。

In [6]:
request_id = table['result'][0]['request_id']
request_id

'20109216_1823419'

然后，就可以通过`request_id`查询Excel表格的下载地址了。将`request_id`传入`getTableRecognitionResult()`就能获取结果。打印一下结果`result`，就能看到下面的内容。其中`result_data`对应的那个网址就是Excel表格的下载地址。将那个网址复制粘贴到浏览器，就能下载包含识别结果的Excel文件了。这个Excel文件有6张工作表，具体包含与图片中表格内容的对应关系如下。`body`储存表格部分的内容，`header`储存表头的文字，`footer`储存表尾的文字。
![](images\relation.png)

In [7]:
result = client.getTableRecognitionResult(request_id)
print(result)

{'result': {'result_data': 'http://bj.bcebos.com/v1/ai-edgecloud/CE6A642920A048F2AD300F6BEAFA9475.xls?authorization=bce-auth-v1%2Ff86a2044998643b5abc89b59158bad6d%2F2020-05-30T03%3A06%3A26Z%2F172800%2F%2F7bfa6447d47fb9366f36db1b9c4dc3db8a545336ad65320ac91a77922ac209b7', 'ret_msg': '已完成', 'request_id': '20109216_1823419', 'percent': 100, 'ret_code': 3}, 'log_id': 1590807993696156}


以上，单张图片识别测试成功。下面就来完善程序，实现批量识别和Excel文件的自动下载。先获取所有图片的路径，存入`pictures`列表。结果如下。

In [1]:
#获取路径下所有图片文件，并存入列表
import os
work_path = "图片\\"
pictures=[] # 存储文件夹内所有文件的路径（包括子目录内的文件）
for root, dirs, files in os.walk(work_path):
    path = [os.path.join(root, name) for name in files]
    pictures.extend(path)
pictures

['图片\\收货表.jpg', '图片\\论文.jpg']

然后将所有图片逐个传入表格识别接口，获取其请求ID及存有识别结果的Excel文件的下载地址。在提取Excel下载路径之前，需要先判断识别是否完成。这是通过识别结果返回的字典中的`'ret_msg'`对应的值来判断的。只有当它是“已完成”时，才能获得下载链接。此处用了`while`循环，每隔2秒刷新一下状态，直到状态是“已完成”时，才提取链接。然后使用`requests.get()`获取下载链接信息，写入Excel文件。Excel自动命名及下载结果如图。
![](images\files.png)

In [2]:
from aip import AipOcr  #导入AipOcr模块，用于做文字识别
import time #时间模块
import requests #用于HTTP请求

APP_ID = '你申请的'
API_KEY = '你申请的'
SECRET_KEY = '你申请的'
client = AipOcr(APP_ID, API_KEY, SECRET_KEY)

#提交识别请求，并储存所有请求ID
for picture in pictures:
    pic = open(picture,'rb') #以二进制方式打开图片
    img = pic.read() #读取
    table = client.tableRecognitionAsync(img)    #调用表格识别模块
    request_id = table['result'][0]['request_id']
    
    #判断识别是否完成，直到完成才根据请求ID获取Excel下载路径
    result = client.getTableRecognitionResult(request_id)  #通过ID获取识别结果
    while result['result']['ret_msg'] != '已完成': #如果状态是“已完成”，才能获取下载地址
        time.sleep(2) #暂停2秒再刷新
        result = client.getTableRecognitionResult(request_id) #持续刷新，直到满足条件
        
    download_path = result['result']['result_data']
    
    #下载并将Excel文件名设为图片名
    excel_name = picture.split(".")[0] + ".xls" #让excel文件的名字与图片相同
    excel = requests.get(download_path) #抓取下载链接
    file = open(excel_name, 'wb') #新建excel文件
    file.write(excel.content) #写入excel文件并保存

此处拿了两张图片做测试，一张比较清晰，一张相对模糊。清晰的图片测试结果如下：
![](images\res1.png)
相对模糊的图片测试结果如下：
![](images\res2.png)
可见，还是清晰度是关键啊。毕竟菜要好吃，至少原料要过得去嘛。